<a href="https://colab.research.google.com/github/An-Dongsun/Melanoma-Detection/blob/feature%2FDL_model/DL_model_%EC%A0%9C%EC%9E%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 흑색종 이미지 분류 모델

- 지루성 각화증 → 387개
- 모반 → 8,570개
- 흑색종(양성) → 5,515개
- 흑색종(악성) → 5,119개
- 기저 세포 암종(BCC) → 516개

In [ ]:
# 필요할 라이브러리 불러오기
import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt

# 전이학습 모델
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications import EfficientNetB7

# 분류기 제작
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Activation, Dropout
from tensorflow.keras.models import Model, Sequential, load_model

# 1. 데이터 불러오기

In [2]:
# 시드를 고정한다.
np.random.seed(42)
tf.random.set_seed(42)

In [3]:
# 이미지 파일들 불러오기
Dataset_directory = '/content/drive/MyDrive/CP1 데이터'

train = tf.keras.preprocessing.image_dataset_from_directory(
        Dataset_directory,
        labels = "inferred",
        label_mode = 'categorical', # 레이블이 범주형 벡터로 인코딩됨을 의미
        class_names = {'BCC(기저 세포 암종)', 'benign(흑색종 양성)', 'malignant(흑색종 악성)', 'nevus(모반)', 'seborrheic_keratosis(지루성 각화증)'},
        color_mode = "rgb",
        batch_size = 128,
        image_size = (256, 256),
        shuffle = True,
        seed = 42,
        validation_split = 0.1,
        subset = 'training',
        interpolation = "bilinear",
        follow_links = False,
        crop_to_aspect_ratio = False,
)

test = tf.keras.preprocessing.image_dataset_from_directory(
        Dataset_directory,
        labels = "inferred",
        label_mode = 'categorical',
        class_names = {'BCC(기저 세포 암종)', 'benign(흑색종 양성)', 'malignant(흑색종 악성)', 'nevus(모반)', 'seborrheic_keratosis(지루성 각화증)'},
        color_mode = "rgb",
        batch_size = 128,
        image_size = (256, 256),
        shuffle = True,
        seed = 42,
        validation_split = 0.1,
        subset = 'validation',
        interpolation = "bilinear",
        follow_links = False,
        crop_to_aspect_ratio = False,
)

Found 20053 files belonging to 5 classes.
Using 18048 files for training.
Found 20053 files belonging to 5 classes.
Using 2005 files for validation.


# Restnet50 전이학습


In [4]:
resnet = ResNet50(weights = 'imagenet', include_top = False)

In [5]:
# ResNet50 레이어들의 파라미터를 학습하도록 설정
for layer in resnet.layers:
    layer.trainable = True

In [6]:
# 모델에 출력층을 생성해준다.
x = resnet.output
x = GlobalAveragePooling2D()(x)
# 다중 분류 모델을 만들어준다.
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(5, activation='softmax')(x) # 출력층을 설계한다.
resnet_model = Model(resnet.input, predictions)

In [7]:
# 모델의 전체적인 형태 확인해보기
resnet_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, None, None,   0           ['input_1[0][0]']                
                                3)                                                                
                                                                                                  
 conv1_conv (Conv2D)            (None, None, None,   9472        ['conv1_pad[0][0]']              
                                64)                                                           

In [8]:
# 모델 컴파일
resnet_model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
# 학습시킨 데이터를 저장시키기 위한 코드다. 
checkpoint_filepath = "resnet_model_best.hdf5"

# early stopping
# val_loss 모니터링, patience = 10 → 연속적으로 10번 값이 나아지지 않으면 조기 종료
early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 10, verbose = 1)

# Validation Set을 기준으로 가장 최적의 모델을 찾기
save_best = keras.callbacks.ModelCheckpoint(filepath = checkpoint_filepath,
                                            monitor = 'val_loss',
                                            verbose = 1,
                                            save_best_only = True,
                                            save_weights_only = True,
                                            mode = 'auto',
                                            save_freq = 'epoch',
                                            options = None)

# 변수 설정을 따로 하는 방법을 적용하기 위한 코드입니다. 
batch_size = 128
epochs_max = 50

# 모델 학습
# 위에서 설정한 batch_size, epochs_max 그대로 가져와서 사용하기
results = resnet_model.fit(train,
                            batch_size = batch_size,
                            epochs = epochs_max,
                            verbose = 1, 
                            validation_data = test,
                            callbacks = [early_stop, save_best])

Epoch 1/50
141/141 [==============================] - ETA: 0s - loss: 0.3634 - accuracy: 0.8719
Epoch 1: val_loss improved from inf to 5.23827, saving model to resnet_model_best.hdf5
141/141 [==============================] - 286s 2s/step - loss: 0.3634 - accuracy: 0.8719 - val_loss: 5.2383 - val_accuracy: 0.3830
Epoch 2/50
141/141 [==============================] - ETA: 0s - loss: 0.2057 - accuracy: 0.9172
Epoch 2: val_loss did not improve from 5.23827
141/141 [==============================] - 280s 2s/step - loss: 0.2057 - accuracy: 0.9172 - val_loss: 15.5683 - val_accuracy: 0.5860
Epoch 3/50
141/141 [==============================] - ETA: 0s - loss: 0.1980 - accuracy: 0.9252
Epoch 3: val_loss improved from 5.23827 to 3.21583, saving model to resnet_model_best.hdf5
141/141 [==============================] - 286s 2s/step - loss: 0.1980 - accuracy: 0.9252 - val_loss: 3.2158 - val_accuracy: 0.4279
Epoch 4/50
141/141 [==============================] - ETA: 0s - loss: 0.1647 - accuracy: 0

## 학습된 모델을 이용하여 테스트 & 저장

In [ ]:
# 학습된 모델을 이용하여 테스트하는 코드
test_loss, test_acc = resnet_model.evaluate(test, verbose = 1)

In [ ]:
acc = results.history['accuracy']
val_acc = results.history['val_accuracy']

loss = results.history['loss']
val_loss = results.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
# 최고의 성능을 내는 파라미터를 불러온다.
resnet_model.load_weights(checkpoint_filepath)

# best model을 이용한 테스트 데이터 예측 정확도 재확인 코드
test_loss, test_acc = resnet_model.evaluate(test, verbose = 1)

In [ ]:
# Model 저장
resnet_model.save('resnet_base_model.h5')